In [ ]:
import tools
from tools.imports import *
from tools.document_tools import GoogleDocWriteTool
from tools.file_mgmt_tools import MoveFileTool, CreateFolderTool, FolderMovementTool, FileOrganizerTool, ImprovedSearchTool
from tools.miscellaneous_mgmt import GoogleDriveUploadTool, GmailSendPdfTool, GoogleSheetsUpdateTool
from tools.initialize_groq import init_groq

In [ ]:
from langchain.agents import load_tools
my_tools = []
_,llm = init_groq()
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
my_tools.append(
    
    GoogleDocWriteTool()
)

my_tools.append(
    
    GoogleDriveUploadTool(credentials_path)
)

my_tools.append(
    
    GoogleSheetsUpdateTool(credentials_path)
)

my_tools.append(
    GmailSendPdfTool(credentials_path)
)

my_tools.append(
    MoveFileTool(credentials_path)
)

my_tools.append(
    CreateFolderTool(credentials_path)
)

my_tools.append(
    FolderMovementTool(credentials_path)
)

my_tools.append(
    FileOrganizerTool(credentials_path)
)

my_tools.append(
    ImprovedSearchTool(credentials_path)
)
humantool = load_tools(
    ["human"],
    llm=llm,
)[0]
my_tools.append(humantool)

In [ ]:
my_tools

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import (
  load_tools, create_structured_chat_agent, AgentExecutor
)
from langchain.agents.structured_chat.prompt import FORMAT_INSTRUCTIONS, PREFIX, SUFFIX
import tools.initialize_groq
from langchain.agents import StructuredChatAgent

_,llm = init_groq()

llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

template = """
RESPOND IN A CLEAR CUT MANNER.
ALWAYS USE THIS FORMAT 

DO NOT SAY THINGS LIKE - 'here is the user input' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
DO NOT ASSUME USER WANTS TO DO ANYTHING AT ALL UNLESS YOU ARE 100% SURE!!!!! UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
BE REALLY CAREFUL WITH FILE AND FOLDER ID'S! ANY WRONG ID'S WILL RESULT IN FAILURE OF OPERATIONS

You need credentials of the user. Here they are:
Name: {name}
Email: {email}
Recipient Email: {recipient_email}
Phone: {phone}
-------
HERE IS PATH to documents: {thepath}
-------
YOU HAVE ACCESS TO THE FOLLOWING TOOLS

{tool_names}
{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}}}
```

-------
HERE IS GUIDANCE ON EXACTLY WHAT TO PASS IN TO THE TOOLS:
GoogleDocWriteTool: INPUT EVERYTHING CORRECTLY EVEN IF ITS LONG!!!!!!!! INPUT ONLY WHAT YOURE GIVEN ! NO CREATIVITIES!!!! append-->true or false lowercase only!!! NO DICTIONARIES, JUST EVERYTHING PASSED AS IT ISSS!!!!!--> INPUT_TEXT: generate it yourself , APPEND: decide whether or not you want to append to document (must be in lowercase)
MoveFileTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FILE_NAME please deduce from the user input and then pass it into tool; NEVER PASS IN 'root' UNLESS USER INDICATES to move file into 'google drive' or 'my drive'; for rest of the parameters, there will be instructions given over time for you.
GoogleSheetsUpdateTool: Input ONLY ONLY these into the tool PROPERLY! --> NAME AND PHONE --> {name}, {phone}
CreateFolderTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER NAME: the folder name as specified in user input; pass in 'root' as the value for parent_folder_id parameter if user indicates 'google drive' or 'my drive' ||| PLEASE OUTPUT THE EXACT NAME OF THE FOLDER THAT WAS CREATED
FolderMovementTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER_NAME please deduce from the user input and then pass it into tool; NEW_PARENT_FOLDER_NAME also deduce from user input and PASS IT IN. SPECIAL CASE - PASS IN 'root' if user says move into google drive or my drive. The IDs you will find out later. for now they are optional.
FileOrganizerTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> PARENT_FOLDER_NAME please deduce from the user input and then pass it into tool - PASS IN 'root' if user says move into google drive or my drive
GoogleDriveUploadTool: INPUTS --> FILE_PATH: {thepath} & USER_EMAIL: {email}
GmailSendPdfTool: IF THERE IS A FILE TO SEND, INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {email}, RECIPIENT EMAIL: {recipient_email}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: {thepath}
GmailSendPdfTool: IF THERE IS NO FILE TO SEND, INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {email}, RECIPIENT EMAIL: {recipient_email}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: nothing
ImprovedSearchTool: DEDUCE THE FILE NAME OR FOLDER NAME from user input, and pass it in to this tool; AND if you know the ID you can pass it in as well.

IMPORTANT: IF YOU REMEMBER YOU HAVE USED A TOOL ALREADY (REFER TO CHAT HISTORY), AND YOU ARE IN THE PROCESS OF USING IT AGAIN, \
YOU MUST ASK USER IF THEY ARE SURE TO USE THE TOOL AGAIN OR NOT.

IMPORTANT: 1. THIS IS FOR ANYTHING INVOLVING FILES OR FOLDERS:
    - MAKE SURE THAT ID'S ARE 100% CORRECT! IF ID'S ARE WRONG, LIFE WILL END AS WE KNOW IT!
   - First, use the ImprovedSearchTool to find the file ID for the file name provided.
   - Then, use the ImprovedSearchTool to find the folder ID for the folder name provided.
   - Once you have both IDs, use the MoveFileTool to move the file to the specified folder.

If user tells you to do something that is not one of these tools/operations,\
      you kindly say that you don't have access to that functionality.

Begin!
---------
Chat History: {chat_history}
User Input: {input}
Thought: {agent_scratchpad}
"""

prompt = ChatPromptTemplate.from_template(template)

agent = create_structured_chat_agent(llm=llm, tools=my_tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=my_tools, handle_parsing_errors=True, verbose=True)

credentials = {
    "name": "Sriram Nallani",
    "email": "sriramnallani35@gmail.com",
    "recipient_email": "thegreatchrisjohn@gmail.com",
    "phone": "4973993399"
}

chat_history = []

tool_strings = []
for tool in my_tools:
    args_schema = re.sub("}", "}}", re.sub("{", "{{", str(tool.args)))
    tool_strings.append(f"{tool.name}: {tool.description}, args: {args_schema}")

print(tool_strings)

result = agent_executor.invoke({
    "input": """what is all my credentials""",
    "chat_history": chat_history,
    "tools": tool_strings,
    "tool_names": ", ".join([tool.name for tool in my_tools]),
    "name": credentials['name'],
    "email": credentials['email'],
    "recipient_email": credentials['recipient_email'],
    "phone": credentials['phone'],
    "thepath": "C:\\Users\\pc-user1\\Downloads\\Okati okati okati.mp3"
})

print(result['output'])
